In [1]:
import numpy as np
import pandas as pd

import sklearn as sk
import matplotlib.pyplot as plt
from obspy import Stream, Trace, UTCDateTime
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

import pylab as pl
from keras.callbacks import ModelCheckpoint

import keras

from keras.layers import Activation, Conv1D, Dense, Dropout, Flatten, MaxPooling3D, BatchNormalization, InputLayer, LSTM
from keras.layers import Dropout
from keras.layers.advanced_activations import LeakyReLU
from keras.losses import categorical_crossentropy
from keras.models import Sequential
from keras.utils import Sequence
from keras.optimizers import Adam
from tensorflow.keras import regularizers
from keras.utils import np_utils
from keras.utils.vis_utils import plot_model
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import ParameterGrid
import re
from sklearn.metrics import confusion_matrix

import os
import sys
classes_dir = 'F:\Thesis_ssd\MasterThesis3.0'
os.chdir(classes_dir)
from Classes.DataProcessing.LoadData import LoadData
from Classes.DataProcessing.BaselineHelperFunctions import BaselineHelperFunctions
from Classes.DataProcessing.DataHandler import DataHandler
from Classes.DataProcessing.DataGenerator import DataGenerator
from Classes.DataProcessing.NoiseAugmentor import NoiseAugmentor
from Classes.Modeling.Models import Models
from Classes.Modeling.RandomGridSearch import RandomGridSearch
from Classes.Modeling.CustomCallback import CustomCallback
from Classes.Scaling.ScalerFitter import ScalerFitter
from Classes.Scaling.MinMaxScalerFitter import MinMaxScalerFitter
from Classes.Scaling.StandardScalerFitter import StandardScalerFitter
from Classes import Tf_shutup
Tf_shutup.Tf_shutup()

from livelossplot import PlotLossesKeras

import tensorflow as tf

from matplotlib.colors import ListedColormap
%matplotlib inline
plt.rcParams["figure.figsize"]= (15,15)
helper = BaselineHelperFunctions()

import sys
ISCOLAB = 'google.colab' in sys.modules

import random
import pprint

base_dir = 'F:\Thesis_ssd\MasterThesis3.0'
os.chdir(base_dir)

In [2]:
### Data conditions: ###
load_args = {
    'earth_explo_only' : True,
    'noise_earth_only' : False,
    'downsample' : True,
    'upsample' : True,
    'frac_diff' : 0.5,
    'seed' : 1,
    'subsample_size' : 0.05
}

loadData = LoadData(**load_args)

train_ds, val_ds, test_ds = loadData.get_datasets()
handler = DataHandler(loadData)
data_gen = DataGenerator(loadData)

In [3]:
print(len(train_ds), len(val_ds), len(test_ds))
classes, counts = handler.get_class_distribution_from_ds(train_ds)
print("Nr noise samples " + len(loadData.noise_ds))

5199 458 458
Total: 5199, earthquake: 2615, explosion: 2584
2635


In [4]:
############ Model picker #############
model_nr = 7
num_classes = len(classes)

########### Hyperparameters ###########
batch_size = 64
epochs = 80
learning_rate = 0.001
#opt = tf.keras.optimizers.SGD(learning_rate=learning_rate, clipnorm=1.0, clipvalue=0.5)
opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
activation = 'tanh'
output_layer_activation = 'softmax'
dropout_rate = 0
filters = 13
kernel_size = 5
l1_r = 0.0001
l2_r = 0.01
padding = 'same'
start_neurons = 32

########### Preprocessing ###########
test = False
use_noise_augmentor = True
detrend = False
use_scaler = False
use_highpass = False
highpass_freq = 0.2

use_tensorboard = False
use_livelossplot = True
use_custom = False

In [5]:
def clear_tensorboard_dir():
    import os
    import shutil
    path = f"{base_dir}/Tensorboard_dir/fit"
    files = os.listdir(path)
    print(files)
    for f in files:
        shutil.rmtree(os.path.join(path,f))
        
if use_tensorboard:
    import datetime
    clear_tensorboard_dir()
    %load_ext tensorboard
    %tensorboard --logdir tensorboard_dir/fit
    log_dir = f"{base_dir}/tensorboard_dir/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
    callbacks = [tensorboard_callback]

if use_custom:
    custom_callback = CustomCallback(data_gen)
    callbacks = custom_callback
elif use_livelossplot:
    callbacks = PlotLossesKeras()

In [6]:
num_ds, channels, timesteps = data_gen.get_trace_shape_no_cast(train_ds)
input_shape = (batch_size, channels, timesteps)

build_model_args = {'model_nr' : model_nr,
                    'input_shape' : input_shape,
                    'num_classes' : num_classes,
                    'dropout_rate' : dropout_rate,
                    'activation' : activation,
                    'output_layer_activation' : output_layer_activation,
                    'l2_r' : l2_r,
                    'l1_r' : l1_r,
                    'full_regularizer' : True,
                    'start_neurons' : start_neurons,
                    'filters' : filters,
                    'kernel_size' : kernel_size,
                    'padding' : 'same'}
model = Models(**build_model_args).model

model_args = {'loss' : "binary_crossentropy",
              'optimizer' : opt,
              'metrics' : ["accuracy","MSE",
                           tf.keras.metrics.Precision(thresholds=None, top_k=None, class_id=None, name=None, dtype=None),
                           tf.keras.metrics.Recall(thresholds=None, top_k=None, class_id=None, name=None, dtype=None)]}
model.compile(**model_args)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (64, 3, 32)               1233152   
_________________________________________________________________
dropout (Dropout)            (64, 3, 32)               0         
_________________________________________________________________
batch_normalization (BatchNo (64, 3, 32)               128       
_________________________________________________________________
lstm_1 (LSTM)                (64, 16)                  3136      
_________________________________________________________________
dropout_1 (Dropout)          (64, 16)                  0         
_________________________________________________________________
batch_normalization_1 (Batch (64, 16)                  64        
_________________________________________________________________
flatten (Flatten)            (64, 16)                  0

In [7]:
scaler = None
if use_scaler:
    scaler = StandardScalerFitter(train_ds).fit_scaler(test = test, detrend = detrend)
aug = None
if use_noise_augmentor:
    aug = NoiseAugmentor(train_ds, use_scaler, scaler, loadData)
    
    

gen_args = {
    'batch_size' : batch_size,
    'test' : test,
    'detrend' : detrend,
    'use_scaler' : use_scaler,
    'scaler' : scaler,
    'use_noise_augmentor' : use_noise_augmentor,
    'augmentor' : aug,
    'num_classes' : num_classes,
    'use_highpass' : use_highpass,
    'highpass_freq' : highpass_freq
}


train_gen = data_gen.data_generator(train_ds, **gen_args)
val_gen = data_gen.data_generator(val_ds, **gen_args)
test_gen = data_gen.data_generator(test_ds, **gen_args)



args = {'steps_per_epoch' : helper.get_steps_per_epoch(train_ds, batch_size, test),
        'epochs' : epochs,
        'validation_data' : val_gen,
        'validation_steps' : helper.get_steps_per_epoch(val_ds, batch_size, test),
        'verbose' : 1,
        'use_multiprocessing' : False, 
        'workers' : 1,
        'callbacks' : [callbacks]
}

model_fit = model.fit(train_gen, **args)

Epoch 1/80ise progress: [------------------> ] 99 %
59/81 [====================>.........] - ETA: 24s - loss: 6.0760 - accuracy: 0.5117 - MSE: 0.3157 - precision: 0.5117 - recall: 0.5117

KeyboardInterrupt: 